In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
import joblib
from bs4 import BeautifulSoup
import requests
from fuzzywuzzy import process

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fifa-22-complete-player-dataset/female_players_17.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_20.csv
/kaggle/input/fifa-22-complete-player-dataset/players_15.csv
/kaggle/input/fifa-22-complete-player-dataset/players_16.csv
/kaggle/input/fifa-22-complete-player-dataset/Career Mode female player datasets - FIFA 16-22.xlsx
/kaggle/input/fifa-22-complete-player-dataset/players_17.csv
/kaggle/input/fifa-22-complete-player-dataset/players_18.csv
/kaggle/input/fifa-22-complete-player-dataset/players_19.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_16.csv
/kaggle/input/fifa-22-complete-player-dataset/players_22.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_18.csv
/kaggle/input/fifa-22-complete-player-dataset/players_21.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_19.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_22.csv
/kaggle/input/fifa-22-complete-player-dataset/female_players_21.cs

In [2]:
matches = pd.read_csv('../input/fifa-world-cup/WorldCupMatches.csv')
matches

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#replace German DR and Germany FR by Germany
#replace Soviet Union by Russia
def replace_name(df):
    if(df['Home Team Name'] in ['German DR', 'Germany FR']):
        df['Home Team Name'] = 'Germany'
    elif(df['Home Team Name'] == 'Soviet Union'):
        df['Home Team Name'] = 'Russia'
    elif(df['Home Team Name'] == "C�te d'Ivoire"):
        df['Home Team Name'] = "Côte d'Ivoire"
        
    if(df['Away Team Name'] in ['German DR', 'Germany FR']):
        df['Away Team Name'] = 'Germany'
    elif(df['Away Team Name'] == 'Soviet Union'):
        df['Away Team Name'] = 'Russia'
    elif(df['Away Team Name'] == "C�te d'Ivoire"):
        df['Away Team Name'] = "Côte d'Ivoire"
        
    return df
    
matches = matches.apply(replace_name, axis='columns')
matches = matches.dropna()
matches

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,05 Jul 2014 - 17:00,Quarter-finals,Arena Fonte Nova,Salvador,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),51179.0,0.0,0.0,Ravshan IRMATOV (UZB),RASULOV Abduxamidullo (UZB),KOCHKAROV Bakhadyr (KGZ),255953.0,300186488.0,NED,CRC
848,2014.0,08 Jul 2014 - 17:00,Semi-finals,Estadio Mineirao,Belo Horizonte,Brazil,1.0,7.0,Germany,,58141.0,0.0,5.0,RODRIGUEZ Marco (MEX),TORRENTERA Marvin (MEX),QUINTERO Marcos (MEX),255955.0,300186474.0,BRA,GER
849,2014.0,09 Jul 2014 - 17:00,Semi-finals,Arena de Sao Paulo,Sao Paulo,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),63267.0,0.0,0.0,C�neyt �AKIR (TUR),DURAN Bahattin (TUR),ONGUN Tarik (TUR),255955.0,300186490.0,NED,ARG
850,2014.0,12 Jul 2014 - 17:00,Play-off for third place,Estadio Nacional,Brasilia,Brazil,0.0,3.0,Netherlands,,68034.0,0.0,2.0,HAIMOUDI Djamel (ALG),ACHIK Redouane (MAR),ETCHIALI Abdelhak (ALG),255957.0,300186502.0,BRA,NED


In [4]:
# drop all rows before 1998
matches = matches[matches.Year >= 1998]
matches

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
516,1998.0,10 Jun 1998 - 17:30,Group A,Stade de France,Saint-Denis,Brazil,2.0,1.0,Scotland,,80000.0,1.0,1.0,GARCIA ARANDA Jose Maria (ESP),TRESACO GRACIA Fernando (ESP),ARANGO Jorge Luis (COL),1014.0,4000.0,BRA,SCO
517,1998.0,10 Jun 1998 - 21:00,Group A,La Mosson,Montpellier,Morocco,2.0,2.0,Norway,,29800.0,1.0,1.0,ANPRASERT Pirom (THA),ABDUL HAMID Halim (MAS),WICKRAMATUNGA Nimal (SRI),1014.0,8725.0,MAR,NOR
518,1998.0,11 Jun 1998 - 17:30,Group B,Stade du Parc Lescure,Bordeaux,Italy,2.0,2.0,Chile,,31800.0,1.0,1.0,BOUCHARDEAU Lucien (NIG),DANTE Dramane (MLI),MANSRI Mohamed (TUN),1014.0,8726.0,ITA,CHI
519,1998.0,11 Jun 1998 - 21:00,Group B,Stade Municipal,Toulouse,Cameroon,1.0,1.0,Austria,,33500.0,0.0,0.0,GONZALEZ CHAVEZ Epifanio (PAR),GALVAN Celestino (PAR),SALINAS Reynaldo (HON),1014.0,8727.0,CMR,AUT
520,1998.0,12 Jun 1998 - 14:30,Group D,La Mosson,Montpellier,Paraguay,0.0,0.0,Bulgaria,,29800.0,0.0,0.0,ALZEID Abdulrahman (KSA),SALIE Achmat (RSA),GHADANFARI Hussain (KUW),1014.0,8728.0,PAR,BUL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,05 Jul 2014 - 17:00,Quarter-finals,Arena Fonte Nova,Salvador,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),51179.0,0.0,0.0,Ravshan IRMATOV (UZB),RASULOV Abduxamidullo (UZB),KOCHKAROV Bakhadyr (KGZ),255953.0,300186488.0,NED,CRC
848,2014.0,08 Jul 2014 - 17:00,Semi-finals,Estadio Mineirao,Belo Horizonte,Brazil,1.0,7.0,Germany,,58141.0,0.0,5.0,RODRIGUEZ Marco (MEX),TORRENTERA Marvin (MEX),QUINTERO Marcos (MEX),255955.0,300186474.0,BRA,GER
849,2014.0,09 Jul 2014 - 17:00,Semi-finals,Arena de Sao Paulo,Sao Paulo,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),63267.0,0.0,0.0,C�neyt �AKIR (TUR),DURAN Bahattin (TUR),ONGUN Tarik (TUR),255955.0,300186490.0,NED,ARG
850,2014.0,12 Jul 2014 - 17:00,Play-off for third place,Estadio Nacional,Brasilia,Brazil,0.0,3.0,Netherlands,,68034.0,0.0,2.0,HAIMOUDI Djamel (ALG),ACHIK Redouane (MAR),ETCHIALI Abdelhak (ALG),255957.0,300186502.0,BRA,NED


In [5]:
# one-hot encode teams

# matches_onehot = matches.copy()
# matches_onehot
# matches_onehot = matches_onehot.drop(['Datetime', 'Stadium', 'Referee', 'Assistant 1', 'Assistant 2', 'RoundID','Win conditions',
#              'Home Team Initials', 'Away Team Initials', 'Half-time Home Goals', 'Half-time Away Goals',
#              'Attendance', 'City', 'MatchID', 'Stage'], 1)

# #find who won: Home win: 1, Away win: 2, Draw: 0
# matches_onehot['Winner'] = '-'

# def find_winner(df):
#     if(int(df['Home Team Goals']) == int(df['Away Team Goals'])):
#         df['Winner'] = 0
#     elif(int(df['Home Team Goals']) > int(df['Away Team Goals'])):
#         df['Winner'] = 1
#     else:
#         df['Winner'] = 2
#     return df

# matches_onehot = matches_onehot.apply(find_winner, axis='columns')

# # Get one hot encoding of column Home Team Name
# one_hot_1 = pd.get_dummies(matches_onehot['Home Team Name'])
# # Drop column B as it is now encoded
# matches_onehot = matches_onehot.drop('Home Team Name',axis = 1)
# # Join the encoded df
# matches_onehot = matches_onehot.join(one_hot_1)
# matches_onehot

# one_hot_2 = pd.get_dummies(matches_onehot['Away Team Name'])
# matches_onehot = matches_onehot.drop('Away Team Name',axis = 1)
# matches_onehot = matches_onehot.join(one_hot_2)
# matches_onehot

In [6]:
#create a dictionary of football team
team_name = {}
index = 0
for idx, row in matches.iterrows():
    name = row['Home Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
    name = row['Away Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
        
team_name

{'Brazil': 0,
 'Scotland': 1,
 'Morocco': 2,
 'Norway': 3,
 'Italy': 4,
 'Chile': 5,
 'Cameroon': 6,
 'Austria': 7,
 'Paraguay': 8,
 'Bulgaria': 9,
 'Saudi Arabia': 10,
 'Denmark': 11,
 'France': 12,
 'South Africa': 13,
 'Spain': 14,
 'Nigeria': 15,
 'Korea Republic': 16,
 'Mexico': 17,
 'Netherlands': 18,
 'Belgium': 19,
 'Argentina': 20,
 'Japan': 21,
 'Yugoslavia': 22,
 'Iran': 23,
 'Jamaica': 24,
 'Croatia': 25,
 'England': 26,
 'Tunisia': 27,
 'Romania': 28,
 'Colombia': 29,
 'Germany': 30,
 'USA': 31,
 'Senegal': 32,
 'Uruguay': 33,
 'rn">Republic of Ireland': 34,
 'Sweden': 35,
 'Slovenia': 36,
 'Turkey': 37,
 'Ecuador': 38,
 'China PR': 39,
 'Costa Rica': 40,
 'Poland': 41,
 'Russia': 42,
 'Portugal': 43,
 'rn">Trinidad and Tobago': 44,
 "Côte d'Ivoire": 45,
 'rn">Serbia and Montenegro': 46,
 'Angola': 47,
 'Australia': 48,
 'Czech Republic': 49,
 'Ghana': 50,
 'Togo': 51,
 'Switzerland': 52,
 'Ukraine': 53,
 'IR Iran': 54,
 'Greece': 55,
 'Algeria': 56,
 'Serbia': 57,
 'New Z

In [7]:
#drop unecessary columns
dropped_matches = matches.drop(['Datetime', 'Stadium', 'Referee', 'Assistant 1', 'Assistant 2', 'RoundID','Win conditions',
             'Home Team Initials', 'Away Team Initials', 'Half-time Home Goals', 'Half-time Away Goals',
             'Attendance', 'City', 'MatchID', 'Stage'], 1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [8]:
#find who won: Home win: 1, Away win: 2, Draw: 0
dropped_matches['Winner'] = '-'

def find_winner(df):
    if(int(df['Home Team Goals']) == int(df['Away Team Goals'])):
        df['Winner'] = 0
    elif(int(df['Home Team Goals']) > int(df['Away Team Goals'])):
        df['Winner'] = 1
    else:
        df['Winner'] = 2
    return df

dropped_matches = dropped_matches.apply(find_winner, axis='columns')

dropped_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner
516,1998.0,Brazil,2.0,1.0,Scotland,1
517,1998.0,Morocco,2.0,2.0,Norway,0
518,1998.0,Italy,2.0,2.0,Chile,0
519,1998.0,Cameroon,1.0,1.0,Austria,0
520,1998.0,Paraguay,0.0,0.0,Bulgaria,0
...,...,...,...,...,...,...
847,2014.0,Netherlands,0.0,0.0,Costa Rica,0
848,2014.0,Brazil,1.0,7.0,Germany,2
849,2014.0,Netherlands,0.0,0.0,Argentina,0
850,2014.0,Brazil,0.0,3.0,Netherlands,2


In [9]:
#replace team name by id in team_name dictionary

def replace_team_name_by_id(df):
    df['Home Team Name'] = team_name[df['Home Team Name']]
    df['Away Team Name'] = team_name[df['Away Team Name']]
    #df['Winner'] = team_name[df['Winner']]
    return df

teamid_matches = dropped_matches.apply(replace_team_name_by_id, axis='columns')
teamid_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner
516,1998.0,0,2.0,1.0,1,1
517,1998.0,2,2.0,2.0,3,0
518,1998.0,4,2.0,2.0,5,0
519,1998.0,6,1.0,1.0,7,0
520,1998.0,8,0.0,0.0,9,0
...,...,...,...,...,...,...
847,2014.0,18,0.0,0.0,40,0
848,2014.0,0,1.0,7.0,30,2
849,2014.0,18,0.0,0.0,20,0
850,2014.0,0,0.0,3.0,18,2


In [10]:
#drop unecessary columns
teamid_matches = teamid_matches.drop(['Year', 'Home Team Goals', 'Away Team Goals'], 1)

teamid_matches

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Home Team Name,Away Team Name,Winner
516,0,1,1
517,2,3,0
518,4,5,0
519,6,7,0
520,8,9,0
...,...,...,...
847,18,40,0
848,0,30,2
849,18,20,0
850,0,18,2


In [11]:
#TRAINING STEPS


#create numpy array
X = teamid_matches.loc[:,['Home Team Name', 'Away Team Name']]
X = np.array(X).astype('float64')

#append data: simply exchange 'home team name' with 'away team name', 'home team championship' with 'away team championship', and replace the result
_X = X.copy()

_X[:,0] = X[:,1]
_X[:,1] = X[:,0]

y = dropped_matches.loc[:,['Winner']]
y = np.array(y).astype('int')
y = np.reshape(y,(1,-1))
y = y[0]


_y = y.copy()

for i in range(len(_y)):
  if(_y[i]==1):
    _y[i] = 2
  elif(_y[i] ==2):
    _y[i] = 1
    
#===========
    
X = np.concatenate((X,_X), axis= 0)

y = np.concatenate((y,_y))

In [12]:
X,y = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(y_train)

[0 1 0 2 1 2 2 2 1 2 0 2 1 0 2 2 2 2 1 1 0 1 2 1 2 1 2 2 0 1 2 1 1 1 0 1 2
 0 1 2 2 2 0 1 1 1 2 0 2 2 0 2 1 2 0 2 0 2 1 0 1 1 1 1 2 1 0 2 2 2 1 1 1 2
 0 1 2 2 1 0 1 2 1 0 2 0 1 2 2 0 1 0 0 0 2 2 1 0 1 1 1 2 2 2 2 0 1 0 2 0 1
 2 2 1 0 0 2 0 1 0 2 1 0 1 0 1 2 2 1 1 1 2 0 1 2 1 2 1 1 2 1 1 2 2 2 2 1 1
 1 1 1 0 1 2 1 0 1 2 1 2 0 2 1 2 0 2 1 2 2 1 1 2 2 2 0 1 2 2 0 1 1 0 1 2 1
 2 2 0 1 2 0 0 0 1 1 0 1 2 1 2 0 0 2 1 2 2 2 0 1 1 2 1 0 0 1 1 0 0 0 1 0 2
 2 0 0 1 2 0 2 0 2 2 2 2 0 0 0 0 0 1 2 2 1 2 2 1 1 0 2 2 1 1 1 0 0 1 2 2 0
 0 1 2 1 2 0 1 1 0 1 1 1 2 1 2 2 2 2 1 2 2 0 1 0 0 2 2 2 2 2 0 1 0 1 2 1 0
 2 1 0 1 1 0 2 0 2 2 0 1 1 0 0 1 2 1 0 0 1 2 2 2 1 2 2 1 0 2 2 2 2 1 1 1 1
 1 2 2 1 0 2 2 1 1 1 2 0 2 1 1 2 1 1 0 1 1 2 0 1 2 1 1 2 2 1 2 1 1 2 2 0 0
 1 0 1 1 2 1 2 0 1 0 1 0 1 1 2 0 1 1 2 1 1 2 2 2 1 2 0 0 1 2 1 2 2 0 2 0 0
 2 1 2 1 1 1 1 1 1 2 2 2 1 2 0 1 2 1 0 2 2 2 1 0 2 1 2 2 1 1 2 2 1 0 1 2 0
 2 1 2 0 1 1 0 0 2 1 2 2 1 0 2 2 1 0 1 2 2 0 1 0 0 1 2 2 0 2 2 1 2 2 1 0 2
 1 1 2 1 1 0 2 2 2 1 0 1 

In [13]:
#train by svm

svm_model = SVC(kernel='rbf', class_weight='balanced', probability=True)
svm_model.fit(X, y)

SVC(class_weight='balanced', probability=True)

In [14]:
print("Predicting on the test set")
#t0 = time()
y_pred = svm_model.predict(X_test)
#print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels=range(3)))

Predicting on the test set
              precision    recall  f1-score   support

           0       0.36      0.57      0.44        42
           1       0.60      0.45      0.52        64
           2       0.58      0.51      0.54        61

    accuracy                           0.50       167
   macro avg       0.52      0.51      0.50       167
weighted avg       0.54      0.50      0.51       167

[[24  8 10]
 [23 29 12]
 [19 11 31]]


In [15]:
def prediction(team1, team2):
  id1 = team_name[team1]
  id2 = team_name[team2]

  x = np.array([id1, id2]).astype('float64')
  x = np.reshape(x, (1,-1))
  _y = svm_model.predict_proba(x)[0]

  text = ('Chance for '+team1+' to win '+team2+' is {}\nChance for '+team2+' to win '+team1+' is {}\nChance for '+team1+' and '+team2+' draw is {}').format(_y[1]*100,_y[2]*100,_y[0]*100)
  return y[0], text

In [16]:
#predict match between France and Uruguay
prob1, text1 = prediction('France', 'Uruguay')
print(text1)

Chance for France to win Uruguay is 34.70501656553558
Chance for Uruguay to win France is 41.73413886238636
Chance for France and Uruguay draw is 23.560844572078068


In [17]:
#predict match between Brazil and Belgium
prob2, text2 = prediction('USA','Belgium')
print(text2)

Chance for USA to win Belgium is 40.041418480804886
Chance for Belgium to win USA is 34.8417295434848
Chance for USA and Belgium draw is 25.116851975710304


In [18]:
# test to merge overall tables

df = pd.DataFrame({
                'Team':['Argentina','Argentina','Italy'],
                'Player':['Dybala','Messi','Chellini'],
                'Overall':[90,93,87]
                })


df2 = pd.DataFrame({
    'Team A':['Argentina'],
    'Team B':['Italy'],
    'Overall A':np.nan,
    'Overall B':np.nan
})

teams = ['Argentina','Italy']

sum = {}
count = {}
for index,row in df.iterrows():
    team = row['Team']
    overall = row['Overall']
    
    if(team not in sum.keys()):
        sum[team] = overall
        count[team] = 1
        print(team,count[team])
    else:
        sum[team] += overall
        count[team] += 1
        print(team,count[team])

avg = {}
for i in sum:
    avg[i] = sum[i]/count[i]

for key in avg.keys():
    df2.loc[(df2['Team A'] == key), 'Overall A'] = avg[key]
    df2.loc[(df2['Team B'] == key), 'Overall B'] = avg[key]
print(df2)

Argentina 1
Argentina 2
Italy 1
      Team A Team B  Overall A  Overall B
0  Argentina  Italy       91.5       87.0


In [19]:
# formatting DOB columnn function

def dob_convert(df):
    
    new = df["dob"].str.split(" ", n = 3, expand = True)
    
    # making separate first name column from new data frame
    df["Day"] = new[0]

    for idx, row in df.iterrows():
        row['Day'] = str(row['Day']).zfill(2)
        
    # making separate last name column from new data frame
    df["Month_name"] = new[1]
    
    months = {'January': 1,
              'February': 2,
              'March': 3,
              'April': 4,
              'May': 5,
              'June': 6,
              'July': 7,
              'August': 8,
              'September': 9,
              'October': 10,
              'November': 11,
              'December': 12}
    
    for idx, row in df.iterrows():
        row['Month_name'] = str(months[row['Month_name']]).zfill(2)
    
    df['Month'] = df['Month_name']
    
    df["Year"] = new[2]

    # Dropping old Name columns
    df.drop(columns =["dob",'Month_name'], inplace = True)

    df['dob'] = df["Year"] + "-" + df["Month"].astype(str) + "-" + df["Day"]
    
    df.drop(columns =["Year",'Month','Day'], inplace = True)
    return df


In [20]:
url = 'https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

results = soup.find_all('h3')

count = 0
teams = []

for result in results:
    results2 = result.find_all('span',class_="mw-headline")
    for team in results2:
        count += 1
        if count <= 32:
            teams.append(team.text)

teams_dict = dict.fromkeys(teams)

tables = soup.find_all('table',{'class':"sortable wikitable plainrowheaders"})

count = 0

for table in tables:
    df = pd.read_html(str(table),flavor='bs4')
    df_wiki_players = pd.DataFrame(df[0])
    teams_dict[teams[count]] = df_wiki_players 
    count += 1

dfs_players = []
for team in teams:
    d = {'Team': team, 'Player': list(teams_dict[team].Player), 'dob': list(teams_dict[team]['Date of birth (age)'])}
    df_players = pd.DataFrame(d)
    for idx,row in df_players.iterrows():
        if '(captain)' in row.Player:
            row.Player = row.Player.replace('(captain)','').strip()
    dfs_players.append(df_players)
    
for i in range(len(dfs_players)):
    dfs_players[i] = dob_convert(dfs_players[i])
    

In [21]:
# simplyfying the player dataset with player and overall
players18 = pd.read_csv('../input/fifa-22-complete-player-dataset/players_18.csv')
players18 = players18.loc[:, players18.columns.intersection(['short_name','long_name','dob','overall','nationality_name'])]

# replace korea republic with south korea
def replace_name(df):
    if df['nationality_name'] == 'Korea Republic':
        df['nationality_name'] = 'South Korea'
    return df

# remove players with same date of birth but not in lineup 
def remove_duplicate_dob(df):
    duplicate_in_dob = df.duplicated(subset=['dob'],keep=False)
    if duplicate_in_dob.any():
        df_duplicate = df.loc[duplicate_in_dob]
        df = df.drop_duplicates(subset=['dob'],keep=False)

        THRESHOLD = 90
        best_match = df_duplicate['long_name'].apply(lambda x: process.extractOne(x, df_duplicate['Player'], score_cutoff=THRESHOLD))
        best_match = best_match.to_frame()
        best_match = best_match.dropna()
        for idx, row in best_match.iterrows():
            row['long_name'] = row['long_name'][0]
        df_duplicate['new_name'] = pd.Series(best_match['long_name'])
        df_duplicate = df_duplicate.dropna().drop_duplicates(subset=['long_name'])
        df_duplicate['Player'] = df_duplicate['new_name']

        df = pd.concat([df,df_duplicate])
        df = df.drop(columns=['new_name'])
        return df
    return df


players18 = players18.apply(replace_name, axis='columns')
dfs_player_overall = []

for i in range(len(dfs_players)):
    try:
        current_team_name = dfs_players[i]['Team'][0]
    except:
        current_team_name = dfs_players[i]['Team'][4]

    current_team = dfs_players[i]
    players_temp = players18.loc[players18['nationality_name'] == current_team_name]
    
    df_player_overall = pd.merge(current_team, players_temp, on='dob', how='inner')
    
    #might drop more names
    df_player_overall.drop(columns =["nationality_name"], inplace = True)
    
    df_player_overall = remove_duplicate_dob(df_player_overall)
    
    dfs_player_overall.append(df_player_overall)
    


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (81,82,83,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
